# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [454]:
#having trouble getting gmap module recognized as installed using this as a work-around
import sys 
!{sys.executable} -m pip install gmaps

In [455]:
import sys 
!{sys.executable}  -m jupyter nbextension enable --py gmaps
!{sys.executable}  -m jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [456]:
# Dependencies and Setup

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint
import json 

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [457]:
#import city weather data from Weatherpy

# Study data files
weather_metadata_path = "output_data/weather_df.csv"

# Read the mouse data and the study results
weather_df = pd.read_csv(weather_metadata_path)


# Read weather_df.cvs file and store into Pandas data frame
weather_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Rikitea,-23.12,-134.97,23.22,76,61,7.55,PF,1604944827
1,1,Vaini,-21.20,-175.20,22.00,94,20,2.10,TO,1604944852
2,2,Kapaa,22.08,-159.32,26.00,78,20,8.20,US,1604944657
3,3,Arraial do Cabo,-22.97,-42.03,28.00,78,40,4.10,BR,1604944567
4,4,Yellowknife,62.46,-114.35,-2.22,79,75,1.50,CA,1604944642


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [498]:
 # Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]].astype(float)

#Store humidity data in this list
humidity =  weather_df["Humidity"].astype(float)



# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
#fig.add_layer(heat_layer)



# # Customize the size of the figure
figure_layout = {
    'width': '1200px',
    'height': '700px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
# Add layer
fig.add_layer(heat_layer)

#Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='700px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [491]:
#chopped up dataframe based on "ideal" conditions
custom_weather_df = weather_df.loc[(weather_df['Max Temp']<=29.9) & (weather_df['Max Temp']>=21.1) 
                                   & (weather_df['Wind Speed']< 10) & (weather_df['Cloudiness'] ==0)]

#cleaned up dataframe
custom_weather_df2 = custom_weather_df.reset_index()
del custom_weather_df2['Unnamed: 0']
del custom_weather_df2['index']
custom_weather_df2

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Saint-Philippe,-21.36,55.77,24.00,73,0,4.60,RE,1604944824
1,Najrān,17.49,44.13,24.00,29,0,1.08,SA,1604944876
2,Riyadh,24.69,46.72,26.00,36,0,2.10,SA,1604944793
3,Buala,-8.14,159.59,26.87,80,0,2.68,SB,1604944897
4,Bilma,18.69,12.92,23.33,28,0,3.52,NE,1604944899
5,Yārāda,17.65,83.27,23.00,78,0,1.00,IN,1604944901
6,Mahuva,21.09,71.76,26.41,34,0,4.22,IN,1604944905
7,Anakapalle,17.68,83.02,23.00,78,0,1.00,IN,1604944924
8,Port Hedland,-20.32,118.57,25.00,73,0,5.70,AU,1604944817
9,Mauá,-23.67,-46.46,28.00,54,0,5.70,BR,1604944790


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [492]:
#establish data frame to capture the cloest hotels received
hotel = []

#count for print message below
count = 0

# loop through the custom weather cities.  Used city because I was attempting to capture the
# city name in the iteration to include in any fallout messages..but gave up.
for city in range(len(custom_weather_df2["City"])):
    
    #for some reason I have define these two variables within the loop..don't understand why
    #if I don't, I only get the same hotel record back for all locations
    lat = custom_weather_df2.loc[city]["Lat"]
    lng = custom_weather_df2.loc[city]["Lng"]
    
    #formating the lat and lng for the API request 
    coordinates = f"{lat},{lng}"
    
    #parameters for the API request, base url defined, response format
    params = {"location": coordinates,"types": "lodging","radius": 5000,"key": g_key}
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   
    response = requests.get(base_url, params)
    
    #serializing the json response
    response_data = response.json()
        
    #append successful response to hotel data frame or append failure message in data frame    
    try:
        hotel.append(response_data["results"][0]["name"])
        count += 1
    except:
        hotel.append("No match found")
        print(f'Skipping...No Hotel Found Within 5000 Meters Of Coordinates: {coordinates}')

print(f'-------------------------------')
print(f'     End of Data Retrieval  \n Closest hotel for {count} locations retrieved below:')
print(f'-------------------------------')
        
        
#Add closest hotels to the existing data frame        
custom_weather_df2["Hotel Name"] = hotel

#print frame to verify
custom_weather_df2

Skipping...No Hotel Found Within 5000 Feet Of Coordinates: 13.93,41.69
-------------------------------
     End of Data Retrieval  
 Closest hotel for 14 locations retrieved below:
-------------------------------


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Saint-Philippe,-21.36,55.77,24.00,73,0,4.60,RE,1604944824,"Chambres d'hôte ""La Trinité"""
1,Najrān,17.49,44.13,24.00,29,0,1.08,SA,1604944876,محمد هادي ال فطيح
2,Riyadh,24.69,46.72,26.00,36,0,2.10,SA,1604944793,InterContinental Riyadh
3,Buala,-8.14,159.59,26.87,80,0,2.68,SB,1604944897,Maringe Lagoon Lodge
4,Bilma,18.69,12.92,23.33,28,0,3.52,NE,1604944899,Expeditions Ténére voyages
5,Yārāda,17.65,83.27,23.00,78,0,1.00,IN,1604944901,"D-26/2, Dolphin Hill,"
6,Mahuva,21.09,71.76,26.41,34,0,4.22,IN,1604944905,Hotel Blue Diamond.
7,Anakapalle,17.68,83.02,23.00,78,0,1.00,IN,1604944924,Satya Lodge
8,Port Hedland,-20.32,118.57,25.00,73,0,5.70,AU,1604944817,The Esplanade Hotel
9,Mauá,-23.67,-46.46,28.00,54,0,5.70,BR,1604944790,Hotel Capriccio Mauá


In [522]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row

# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in custom_weather_df2.iterrows()]
locations =custom_weather_df2[["Lat", "Lng"]]



In [523]:
# Add marker layer ontop of heat map

# Assign the marker layer to a variable
#markers = gmaps.marker_layer(locations, info_box_content= [f"Hotel Name: {hotel}" for hotel in hotels])
markers = gmaps.marker_layer(locations, info_box_content= hotel_info)

# Add the layer to the map
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='700px', margin='0 auto 0 auto', padding='1px', wi…